# Dynamic Lot-Size Model

## Setup

In [10]:
%load_ext autoreload
%autoreload 2

### Libraries

In [1]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
import matplotlib.pyplot as plt

In [11]:
from plotly import express as px
from plotly import graph_objects as go

### Configuration

In [12]:
pd.options.plotting.backend = "plotly"

## Formulation

$$
\begin{align}
    \text{min}~~ & \sum_{t \in T}{(h_{t} I_{t} + s_{t} y_{t})} \\
    \text{s.t.}~~ & I_{t} = I_{t - 1} + x_{t} - d_{t} & \forall ~ t \in T; t \geq 2\\
    & I_{1} = I_{0} + x_{1} - d_{1}\\
    & x_{t} \leq M y_{t} & \forall ~ t \in T \\
    & x_{t}; I_{t} \geq 0 & \forall ~ t \in T \\
    & y_{t} \in \left \{ 0, 1 \right \} & \forall ~ t \in T\\
\end{align}
$$

## Data

In [2]:
dataset = pd.read_csv("./input_wagner.csv", index_col=0)
dataset.head()

,setup_cost,inventory_cost,demand
t,,,
1,85,1.0,69
2,102,1.0,29
3,102,1.0,36
4,101,1.0,61
5,98,1.0,61


#### maximum cost as a baseline comparison

In [3]:
max_cost = dataset.setup_cost.sum()
print(f"Maximum cost: {max_cost:.1f}")

Maximum cost: 1234.0


## model

In [4]:
model = pyo.ConcreteModel()

### Set: Planning Horizon

In [5]:
model.T = pyo.Set(initialize=dataset.index.to_list())

### Parameters


In [6]:
model.demand = pyo.Param(model.T, initialize=dataset["demand"])
model.initial_cost = pyo.Param(model.T, initialize=dataset["setup_cost"])
model.holding_cost = pyo.Param(model.T, initialize=dataset["inventory_cost"])

#### Big M: Initial Cost

In [7]:
total_demand = sum(model.demand[:])
model.intial_cost_big_m = pyo.Param(initialize=total_demand)

### Decision variables


In [8]:
model.x = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.y = pyo.Var(model.T, within=pyo.Binary)
model.I = pyo.Var(model.T, within=pyo.NonNegativeReals)

### Constraints

#### Inventory balance *special in the first instant

In [9]:
@model.Constraint(model.T)
def inventory_rule(model, time):
    if time == model.T.first():
        return model.I[time] == model.x[time] - model.demand[time]
    else:
        t_prev = model.T.prev(time)
        return model.I[time] == model.I[t_prev] + model.x[time] - model.demand[time]

In [ ]:
# Indicator constraint activates y in case x is greater than zero
# def active_prod(###, ###):
#     return ###


# model.active_prod = pyo.Constraint(###, rule=active_prod)

In [ ]:
# Define the objective
# def total_holding(###):
#     return ###


# def total_setup(###):
#     return ###


# def total_cost(###):
#     return ###


# model.obj = pyo.Objective(rule=total_cost, sense=###)

## Solution

In [ ]:
solver = pyo.SolverFactory("appsi_highs")

In [ ]:
solver.solve(model, tee=True)

In [ ]:
opt_value = model.obj()
print(f"Best cost {opt_value}")
print(f"% savings {100 * (1 - opt_value / max_cost) :.2f}")

In [ ]:
dataset["production"] = [model.x[t].value for t in dataset.index]
dataset["inventory"] = [model.I[t].value for t in dataset.index]

In [ ]:
fig, ax = plt.subplots(figsize=[6, 3], dpi=100)
x = dataset.index
width = 0.35
ax.bar(x - width/2, dataset.production, width, color="darkgreen", label="production")
ax.bar(x + width/2, dataset.demand, width, color="navy", label="demand")
ax.set_xticks(x)
ax.set_ylabel("Qtd")
ax.set_xlabel("t")
ax.legend()
fig.tight_layout()
plt.show()